<a href="https://colab.research.google.com/github/Sim98B/MushroomVision/blob/main/MushD4/MushDetector4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mushroom Detector

## Setup

In [1]:
# Importing main libraries
try:
  import torch
  import torchvision
  import torchinfo
except:
  print("[INFO] Torch and Torchvision not installed, downloading")
  !pip install -q torch torchvision torchinfo
import torch
print(f"torch: {torch.__version__}")
import torchvision
print(f"torchvision: {torchvision.__version__}")
import torchinfo
print(f"torchinfo: {torchinfo.__version__}")

# Utilities
import os
from pathlib import Path
import requests
import shutil
from timeit import default_timer as timer
import itertools
import warnings
warnings.filterwarnings("ignore")

# Manipulation
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Neural networks
from torch import nn
from torchvision import transforms
import torchvision.models
from torchinfo import summary

# Metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Cloning GitHub repository
!git clone https://github.com/Sim98B/MushroomVision
!mv MushroomVision/Modules/Libraries .

# Custom functions
from Libraries import data, extractors, model_ops, utils

shutil.rmtree("MushroomVision")

[INFO] Torch and Torchvision not installed, downloading
torch: 2.1.0+cu118
torchvision: 0.16.0+cu118
torchinfo: 1.8.0
Cloning into 'MushroomVision'...
remote: Enumerating objects: 64128, done.
remote: Counting objects: 100% (64128/64128), done.
remote: Compressing objects: 100% (63013/63013), done.
remote: Total 64128 (delta 1037), reused 64092 (delta 1012), pack-reused 0
Receiving objects: 100% (64128/64128), 700.74 MiB | 43.18 MiB/s, done.
Resolving deltas: 100% (1037/1037), done.
Updating files: 100% (1739/1739), done.


In [2]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Data: [CIFAR100](https://www.cs.toronto.edu/~kriz/cifar.html)
The best way to identify those photos that contain photos is to use photos of 100 different subjects.

In [43]:
tensor_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform = tensor_transform)
test_data = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform = tensor_transform)

print(f"{len(train_data)} images for train divided into {len(train_data.classes)} classes")
print(f"{len(test_data)} images for train divided into {len(test_data.classes)} classes")

Files already downloaded and verified
Files already downloaded and verified
50000 images for train divided into 100 classes
10000 images for train divided into 100 classes


In [44]:
# We have a mushroom class in this dataset, at index 51
for i in range(len(train_data.classes)):
  if train_data.classes[i] == "mushroom":
    print(f"{i+1}. {train_data.classes[i].upper()}")

class_names = train_data.classes

52. MUSHROOM


## Baseline
As in the [PyTorch example](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) we are going to use a simple convolutional baseline model.

In [55]:
# Creating a simple convolutional model

utils.set_seed()

class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 100)

  def forward(self, x):
    x = self.pool(torch.nn.functional.relu(self.conv1(x)))
    x = self.pool(torch.nn.functional.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = torch.nn.functional.relu(self.fc1(x))
    x = torch.nn.functional.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = Classifier()
model

Classifier(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=100, bias=True)
)

In [58]:
import torch.nn as nn
import torch.nn.functional as F

class ComplexClassifier(nn.Module):
    def __init__(self):
        super(ComplexClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 100)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)  # Adjust the size based on the output size of the last convolution
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Creazione del modello
model = ComplexClassifier()
model

ComplexClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=100, bias=True)
)

In [59]:
# Dataloaders
train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                           batch_size = 32,
                                           shuffle = True,
                                           num_workers = os.cpu_count(),
                                           pin_memory = True)

test_loader = torch.utils.data.DataLoader(dataset = test_data,
                                          batch_size = 32,
                                          shuffle = False,
                                          num_workers = os.cpu_count(),
                                          pin_memory = True)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [60]:
# Trainig the model for 30 epochs
results = model_ops.train(model = model,
                          train_dataloader = train_loader,
                          test_dataloader = test_loader,
                          loss_function = criterion,
                          optimizer = optimizer,
                          metric = "accuracy",
                          epochs = 150,
                          device = device,
                          verbose = 2)

Training:   0%|          | 0/150 [00:00<?, ?it/s]

Epoch 01: Train Loss: 4.605 | Train Accuracy: 1.38% | Test Loss: 4.603 | Test Accuracy: 2.06%
Epoch 02: Train Loss: 4.596 | Train Accuracy: 2.53% | Test Loss: 4.575 | Test Accuracy: 2.34%
Epoch 03: Train Loss: 4.324 | Train Accuracy: 4.73% | Test Loss: 4.052 | Test Accuracy: 8.09%
Epoch 04: Train Loss: 3.904 | Train Accuracy: 10.34% | Test Loss: 3.766 | Test Accuracy: 13.05%
Epoch 05: Train Loss: 3.670 | Train Accuracy: 14.00% | Test Loss: 3.580 | Test Accuracy: 15.97%
Epoch 06: Train Loss: 3.493 | Train Accuracy: 16.95% | Test Loss: 3.449 | Test Accuracy: 17.82%
Epoch 07: Train Loss: 3.338 | Train Accuracy: 19.69% | Test Loss: 3.317 | Test Accuracy: 20.48%
Epoch 08: Train Loss: 3.198 | Train Accuracy: 22.09% | Test Loss: 3.205 | Test Accuracy: 22.56%
Epoch 09: Train Loss: 3.078 | Train Accuracy: 24.12% | Test Loss: 3.125 | Test Accuracy: 23.93%
Epoch 10: Train Loss: 2.966 | Train Accuracy: 26.32% | Test Loss: 3.058 | Test Accuracy: 25.42%
Epoch 11: Train Loss: 2.854 | Train Accuracy: 

KeyboardInterrupt: ignored

Despite the many epochs of learning a simple baseline cannot fully capture the differences between classes. Therefore, we will be 4 pre-trained models, **AlexNet**, **VGG**, **DenseNet** and **ResNet**, using a couple of slice of the whole dataset: a random **10%** and **20%** of the train set.

## Setting up experiments
Our motto is **start simple** and **add complexity then**.

### Getting the 10% and 20% datasets from **CIFAR100**

In [6]:
# Using a StratifiedShuffleSplit to get indices for both slices
from sklearn.model_selection import StratifiedShuffleSplit

# A simple tensor transform
tensor_transform = transforms.Compose([
    transforms.ToTensor(),
])

# Data
dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=tensor_transform)

# Setting up slicers
stratified_split_10_percent = StratifiedShuffleSplit(n_splits = 1, train_size = 0.1, random_state = 42)
stratified_split_20_percent = StratifiedShuffleSplit(n_splits = 1, train_size = 0.2, random_state = 42)

# Indices
indices_10_percent = next(stratified_split_10_percent.split(dataset.data, dataset.targets))
indices_20_percent = next(stratified_split_20_percent.split(dataset.data, dataset.targets))

# New datasets
dataset_10percent = torch.utils.data.Subset(dataset, indices_10_percent[0])
dataset_20percent = torch.utils.data.Subset(dataset, indices_20_percent[0])

len(dataset_10percent), len(dataset_20percent)

Files already downloaded and verified


(5000, 10000)

### Getting ready with 4 features extractors

In [ ]:
ALEXNET, alexnet_transformer = extractors.create_model(model_name = "alexnet", output_shape = len(class_names))
VGG, vgg_transformer = extractors.create_model(model_name = "vgg16", output_shape = len(class_names))
DENSENET, densenet_transformer = extractors.create_model(model_name = "densenet121", output_shape = len(class_names))
RESNET, resnet_transformer = extractors.create_model(model_name = "resnet50", output_shape = len(class_names))

for model in [ALEXNET, VGG, DENSENET, RESNET]:
  if model.__class__.__name__ != "ResNet":
    print(f"{model.__class__.__name__} params:\nTOTAL: {summary(model, input_size = (32, 3, 256, 256)).total_params:,}\nTRAINABLE: {summary(model, input_size = (32, 3, 256, 256)).trainable_params:,}")
  else:
    print(f"{model.__class__.__name__} params:\nTOTAL: {summary(model, input_size = (32, 3, 232, 232)).total_params:,}\nTRAINABLE: {summary(model, input_size = (32, 3, 232, 232)).trainable_params:,}")

AlexNet params:
TOTAL: 57,413,540
TRAINABLE: 409,700
VGG params:
TOTAL: 134,670,244
TRAINABLE: 409,700
DenseNet params:
TOTAL: 7,056,356
TRAINABLE: 102,500
ResNet params:
TOTAL: 23,712,932
TRAINABLE: 204,900


In [7]:
# Downloading test data; they will be the same for both 10% and 20% experiments
test_data = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform = tensor_transform)

Files already downloaded and verified


In [ ]:
utils.set_seed()
experiment_number = 0

models = ["alexnet", "vgg16", "densenet121", "resnet50"]
data_percentages = ["10%", "20%"]
epochs = [5, 10]

for extractor, data, epoch in itertools.product(models, data_percentages, epochs):
  experiment_number += 1
  print(f"Experiment n°{experiment_number}, model: {extractor}, {data} data, {epoch} epochs")

  train_data = dataset_10percent.dataset if data == "10%" else dataset_20percent.dataset

  model, transformer = extractors.create_model(model_name = extractor, output_shape=len(class_names))
  train_data.transform = transformer
  test_data.transform = transformer

  experiment_train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                                        batch_size=32,
                                                        shuffle=True,
                                                        num_workers=os.cpu_count(),
                                                        pin_memory=True)

  experiment_val_loader = torch.utils.data.DataLoader(dataset=test_data,
                                                      batch_size=32,
                                                      shuffle=False,
                                                      num_workers=os.cpu_count(),
                                                      pin_memory=True)

  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

  model_ops.train(model = model,
                  train_dataloader = experiment_train_loader,
                  test_dataloader = experiment_val_loader,
                  loss_function = criterion,
                  optimizer = optimizer,
                  metric = "accuracy",
                  epochs = epoch,
                  device = device,
                  writer = utils.create_writer(experiment_name = f"{epoch}_epochs",
                                               model_name = model.__class__.__name__,
                                               extra = f"{data}_data"),
                  verbose = 1)

Experiment n°1, model: alexnet, 10% data, 5 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-27/5_epochs/AlexNet/10%_data...


Training:   0%|          | 0/5 [00:00<?, ?it/s]

Experiment n°2, model: alexnet, 10% data, 10 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-27/10_epochs/AlexNet/10%_data...


Training:   0%|          | 0/10 [00:00<?, ?it/s]

Experiment n°3, model: alexnet, 20% data, 5 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-27/5_epochs/AlexNet/20%_data...


Training:   0%|          | 0/5 [00:00<?, ?it/s]

Experiment n°4, model: alexnet, 20% data, 10 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-27/10_epochs/AlexNet/20%_data...


Training:   0%|          | 0/10 [00:00<?, ?it/s]

Experiment n°5, model: vgg16, 10% data, 5 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-27/5_epochs/VGG/10%_data...


Training:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs